In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime


# Create a timetable for our interested data

In [2]:
def last_day_of_month(any_day):    
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)   
    return next_month - datetime.timedelta(days=next_month.day)

lst = []
for year in (2019, 2020):
    for month in range(1, 13):
        lst.append(last_day_of_month(datetime.date(year, month, 1)))
            
data = {'Date': lst}
df = pd.DataFrame(data) 

In [3]:
df['Date'] = pd.to_datetime(df['Date'])
#drop rows after September
df = df[:-3]

In [4]:
OxCGRT_latest = pd.read_csv(r"D:\Download\COVID19\OxCGRT_latest1.csv")

In [5]:
OxCGRT_latest['Date'] = pd.to_datetime(OxCGRT_latest['Date'], format='%Y%m%d')

In [6]:
#df1['Date'] = pd.to_datetime(df1['Date'], format='%Y%m%d').dt.strftime('%m/%d/%Y')

In [7]:
aus = OxCGRT_latest.loc[OxCGRT_latest["CountryName"] == "Australia"]

In [8]:
df = df.merge(aus, on='Date', how='left')
df[['CountryName']] = df[['CountryName']].fillna(value="Australia")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 20
Data columns (total 42 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Date                                   21 non-null     datetime64[ns]
 1   CountryName                            21 non-null     object        
 2   CountryCode                            9 non-null      object        
 3   C1_School closing                      9 non-null      float64       
 4   C1_Flag                                7 non-null      float64       
 5   C2_Workplace closing                   9 non-null      float64       
 6   C2_Flag                                7 non-null      float64       
 7   C3_Cancel public events                9 non-null      float64       
 8   C3_Flag                                7 non-null      float64       
 9   C4_Restrictions on gatherings          9 non-null      float64     

# Drop anything not related or duplicated

In [9]:
df = df.drop(['CountryCode', 'C1_Flag', 'C2_Flag', 'C3_Flag', 'C4_Flag', 'C5_Flag', 'C6_Flag', 'C7_Flag', 'E1_Flag', 'H1_Flag', 'M1_Wildcard', 'StringencyIndexForDisplay', 'StringencyLegacyIndexForDisplay', 'GovernmentResponseIndexForDisplay','ContainmentHealthIndexForDisplay', 'EconomicSupportIndexForDisplay'], axis=1)

# Since there was no COVID-19 in 2019, fill NaN to 0

In [10]:
df.isnull().sum()

Date                                      0
CountryName                               0
C1_School closing                        12
C2_Workplace closing                     12
C3_Cancel public events                  12
C4_Restrictions on gatherings            12
C5_Close public transport                12
C6_Stay at home requirements             12
C7_Restrictions on internal movement     12
C8_International travel controls         12
E1_Income support                        12
E2_Debt/contract relief                  12
E3_Fiscal measures                       12
E4_International support                 12
H1_Public information campaigns          12
H2_Testing policy                        12
H3_Contact tracing                       12
H4_Emergency investment in healthcare    12
H5_Investment in vaccines                12
ConfirmedCases                           12
ConfirmedDeaths                          12
StringencyIndex                          12
StringencyLegacyIndex           

In [11]:
df = df.fillna(value=0)

# Check and delete any column that contains only 0

In [12]:
(df.iloc[:, 0:] == 0).all()

Date                                     False
CountryName                              False
C1_School closing                        False
C2_Workplace closing                     False
C3_Cancel public events                  False
C4_Restrictions on gatherings            False
C5_Close public transport                False
C6_Stay at home requirements             False
C7_Restrictions on internal movement     False
C8_International travel controls         False
E1_Income support                        False
E2_Debt/contract relief                  False
E3_Fiscal measures                        True
E4_International support                  True
H1_Public information campaigns          False
H2_Testing policy                        False
H3_Contact tracing                       False
H4_Emergency investment in healthcare     True
H5_Investment in vaccines                 True
ConfirmedCases                           False
ConfirmedDeaths                          False
StringencyInd

In [13]:
df = df.loc[:, (df != 0).any(axis=0)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 20
Data columns (total 22 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Date                                  21 non-null     datetime64[ns]
 1   CountryName                           21 non-null     object        
 2   C1_School closing                     21 non-null     float64       
 3   C2_Workplace closing                  21 non-null     float64       
 4   C3_Cancel public events               21 non-null     float64       
 5   C4_Restrictions on gatherings         21 non-null     float64       
 6   C5_Close public transport             21 non-null     float64       
 7   C6_Stay at home requirements          21 non-null     float64       
 8   C7_Restrictions on internal movement  21 non-null     float64       
 9   C8_International travel controls      21 non-null     float64       
 10  E1_I

# Read target data

In [14]:
table_02 = pd.read_csv(r"D:\Download\COVID19\Table 02. Labour force status by state, territory, greater capital city and rest of state (ASGS) and sex.csv")

In [15]:
table_02

,Unnamed: 0,Australia ; Employed total ; Persons ;,Australia ; Employed total ; > Males ;,Australia ; Employed total ; > Females ;,Australia ; > Employed full-time ; Persons ;,Australia ; > Employed full-time ; > Males ;,Australia ; > Employed full-time ; > Females ;,Australia ; > Employed part-time ; Persons ;,Australia ; > Employed part-time ; > Males ;,Australia ; > Employed part-time ; > Females ;,...,>> Greater Melbourne ; Labour force total ; Persons ;,>> Greater Melbourne ; Labour force total ; > Males ;,>> Greater Melbourne ; Labour force total ; > Females ;,>> Greater Melbourne ; Participation rate ; Persons ;,>> Greater Melbourne ; Participation rate ; > Males ;,>> Greater Melbourne ; Participation rate ; > Females ;,>> Greater Melbourne ; Not in the labour force (NILF) ; Persons ;,>> Greater Melbourne ; Not in the labour force (NILF) ; > Males ;,>> Greater Melbourne ; Not in the labour force (NILF) ; > Females ;,>> Greater Melbourne ; Civilian population aged 15 years and over ; Persons ;
0,Unit,000,000,000,000,000,000,000,000,000,...,000,000,000,Percent,Percent,Percent,000,000,000,000
1,Series Type,Original,Original,Original,Original,Original,Original,Original,Original,Original,...,Original,Original,Original,Original,Original,Original,Original,Original,Original,Original
2,Data Type,STOCK,STOCK,STOCK,STOCK,STOCK,STOCK,STOCK,STOCK,STOCK,...,STOCK,STOCK,STOCK,PERCENT,PERCENT,PERCENT,STOCK,STOCK,STOCK,STOCK
3,Frequency,Month,Month,Month,Month,Month,Month,Month,Month,Month,...,Month,Month,Month,Month,Month,Month,Month,Month,Month,Month
4,Collection Month,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,Apr-2020,12405.1,6579.6,5825.4,8601.3,5383.3,3218.0,3803.8,1196.3,2607.5,...,2772.9,1483.1,1289.8,65.7,71.5,60.0,1450.6,591.3,859.3,4223.6
355,May-2020,12185.7,6476.9,5708.8,8522.8,5338.1,3184.7,3662.9,1138.8,2524.1,...,2739.1,1466.0,1273.0,64.8,70.6,59.2,1487.8,610.2,877.6,4226.8
356,Jun-2020,12387.7,6554.1,5833.6,8496.7,5318.4,3178.3,3891.0,1235.7,2655.3,...,2773.6,1479.3,1294.3,65.6,71.2,60.1,1456.5,598.6,857.9,4230.1
357,Jul-2020,12476.6,6596.9,5879.8,8567.9,5338.9,3229.0,3908.7,1258.0,2650.7,...,2766.6,1472.7,1293.9,65.4,70.8,60.1,1466.4,606.8,859.5,4232.9


In [16]:
#table_02.info(verbose=True)

# Select the column we need and drop unnecessary data

In [17]:
new_table_02 = table_02[["Unnamed: 0", "Australia ;  Employed total ;  Persons ;", "Australia ;  > Employed full-time ;  Persons ;", "Australia ;  > Employed part-time ;  Persons ;", "Australia ;  Unemployment rate ;  Persons ;"]]

In [18]:
new_table_02 = new_table_02.tail(20)

In [19]:
new_table_02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 339 to 358
Data columns (total 5 columns):
 #   Column                                          Non-Null Count  Dtype 
---  ------                                          --------------  ----- 
 0   Unnamed: 0                                      20 non-null     object
 1   Australia ;  Employed total ;  Persons ;        20 non-null     object
 2   Australia ;  > Employed full-time ;  Persons ;  20 non-null     object
 3   Australia ;  > Employed part-time ;  Persons ;  20 non-null     object
 4   Australia ;  Unemployment rate ;  Persons ;     20 non-null     object
dtypes: object(5)
memory usage: 932.0+ bytes


In [20]:
new_table_02['Unnamed: 0'] = pd.to_datetime(new_table_02['Unnamed: 0'], format='%b-%Y').dt.strftime('%b-%Y')
#new_table_02=new_table_02.rename(columns = {'Unnamed: 0':'Date'})
new_table_02.columns = ['Date','Employed_total','Full_time', 'Part_time', 'Unemployment_rate']

In [21]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d').dt.strftime('%b-%Y')

In [22]:
df = df.merge(new_table_02, on='Date', how='left')


In [23]:
df['Employed_total'] = pd.to_numeric(df['Employed_total'])
df['Full_time'] = pd.to_numeric(df['Full_time'])
df['Part_time'] = pd.to_numeric(df['Part_time'])
df['Unemployment_rate'] = pd.to_numeric(df['Unemployment_rate'])

In [24]:
df[['Employed_total','Full_time', 'Part_time', 'Unemployment_rate']] = df[['Employed_total','Full_time', 'Part_time', 'Unemployment_rate']].fillna(value=df[['Employed_total','Full_time', 'Part_time', 'Unemployment_rate']].mean())

In [25]:
df

,Date,CountryName,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,...,ConfirmedDeaths,StringencyIndex,StringencyLegacyIndex,GovernmentResponseIndex,ContainmentHealthIndex,EconomicSupportIndex,Employed_total,Full_time,Part_time,Unemployment_rate
0,Jan-2019,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,12603.100,8738.00,3865.100,5.50
1,Feb-2019,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,12802.200,8828.30,3973.800,5.40
2,Mar-2019,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,12785.200,8763.50,4021.800,5.40
3,Apr-2019,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,12831.900,8746.40,4085.500,5.30
4,May-2019,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,12920.900,8768.70,4152.100,5.10
5,Jun-2019,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,12889.100,8783.00,4106.100,5.10
6,Jul-2019,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,12884.600,8854.60,4030.000,5.20
7,Aug-2019,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,12853.700,8745.30,4108.400,5.30
8,Sep-2019,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,12919.800,8803.10,4116.800,5.10
9,Oct-2019,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,12891.900,8795.10,4096.900,5.00


# Change new cases and deaths to monthly changes

In [26]:
df['NewCasesMonthlyChange'] = df['ConfirmedCases'] - df['ConfirmedCases'].shift()
df['NewDeathsMonthlyChange'] = df['ConfirmedDeaths'] - df['ConfirmedDeaths'].shift()
df = df.fillna(0)

In [27]:
df[['Date','ConfirmedCases','NewCasesMonthlyChange', 'NewDeathsMonthlyChange']]

,Date,ConfirmedCases,NewCasesMonthlyChange,NewDeathsMonthlyChange
0,Jan-2019,0.0,0.0,0.0
1,Feb-2019,0.0,0.0,0.0
2,Mar-2019,0.0,0.0,0.0
3,Apr-2019,0.0,0.0,0.0
4,May-2019,0.0,0.0,0.0
5,Jun-2019,0.0,0.0,0.0
6,Jul-2019,0.0,0.0,0.0
7,Aug-2019,0.0,0.0,0.0
8,Sep-2019,0.0,0.0,0.0
9,Oct-2019,0.0,0.0,0.0


In [28]:
df.to_csv(r"D:\Download\COVID19\association_rule_raw.csv", index = False, header=True)

# Data preprocessing for MOWCATL & DRBARMS 

In [29]:
def assign_label_NewCasesMonthlyChange(i):
    if i >= 3000:
        return 'cases_huge_increase'
    if i >= 600 and i < 3000:
        return 'cases_increase'
    if i >0 and i < 600:
        return 'cases_small_increase'
    if i == 0:
        return 'no_cases'
new_df = pd.DataFrame(df["NewCasesMonthlyChange"].apply(assign_label_NewCasesMonthlyChange))

In [30]:
def assign_label_NewDeathsMonthlyChange(i):
    if i >= 100:
        return 'deaths_huge_increase'
    if i >= 50 and i < 100:
        return 'deaths_increase'
    if i >0 and i < 50:
        return 'deaths_small_increase'
    if i == 0:
        return 'no_deaths'
new_df["NewDeathsMonthlyChange"] = pd.DataFrame(df["NewDeathsMonthlyChange"].apply(assign_label_NewDeathsMonthlyChange))


In [31]:
def assign_label_School_closing(i):
    if i == 3:
        return 'School_close_Required_all'
    if i == 2:
        return 'School_close_Required_partly'
    if i == 1:
        return 'School_close_Recommended'
    if i == 0:
        return 'School_normal'
new_df["School_closing"] = pd.DataFrame(df["C1_School closing"].apply(assign_label_School_closing))

In [32]:
def assign_label_Workplace_closing(i):
    if i == 3:
        return 'Workplace_close_all_essential_only'
    if i == 2:
        return 'Workplace_close_Required_partly'
    if i == 1:
        return 'Workplace_close_Recommended'
    if i == 0:
        return 'Workplace_normal'
new_df["Workplace_closing"] = pd.DataFrame(df["C2_Workplace closing"].apply(assign_label_Workplace_closing))


In [33]:
def assign_label_Cancel_public_events(i):    
    if i == 2:
        return 'public_events_cancel_Required'
    if i == 1:
        return 'public_events_cancel_Recommended'
    if i == 0:
        return 'Public_events_normal'
new_df["Cancel_public_events"] = pd.DataFrame(df["C3_Cancel public events"].apply(assign_label_Cancel_public_events))


In [34]:
def assign_label_gatherings_restrictions(i):  
    if i == 4:
        return 'Restrictions_ten_ppl_less'
    if i == 3:
        return 'Restrictions_tenppl_more'
    if i == 2:
        return 'Restrictions_hundred_ppl_more'
    if i == 1:
        return 'Restrictions_thousand_ppl_more'
    if i == 0:
        return 'No_gatherings_restrictions'
new_df["Gatherings_restrictions"] = pd.DataFrame(df["C4_Restrictions on gatherings"].apply(assign_label_gatherings_restrictions))


In [35]:
def assign_label_Close_public_transport(i):     
    if i == 2:
        return 'transport_closing_require'
    if i == 1:
        return 'transport_closing_recommend'
    if i == 0:
        return 'transport_normal'
new_df["Close_public_transport"] = pd.DataFrame(df["C5_Close public transport"].apply(assign_label_Close_public_transport))


In [36]:
def assign_label_stay_home(i):  
    if i == 3:
        return 'stay_home_required_one_person_only'
    if i == 2:
        return 'stay_home_required_exercise_only'
    if i == 1:
        return 'stay_home_recommend'
    if i == 0:
        return 'stay_home_not_require'
new_df["stay_home_requirements"] = pd.DataFrame(df["C6_Stay at home requirements"].apply(assign_label_stay_home))


In [37]:
def assign_label_movement_restrictions(i):    
    if i == 2:
        return 'restrict_movement'
    if i == 1:
        return 'recommend_movement_restriction'
    if i == 0:
        return 'no_movement_restrictions'
new_df["movement_restrictions"] = pd.DataFrame(df["C7_Restrictions on internal movement"].apply(assign_label_movement_restrictions))


In [38]:
def assign_label_international_travel_controls(i): 
    if i == 4:
        return 'border_closure'
    if i == 3:
        return 'highrisk_regions_ban'
    if i == 2:
        return 'quarantine_arrivals_from_highrisk_regions'
    if i == 1:
        return 'international_arrivals_screening'
    if i == 0:
        return 'no_travel_control'
new_df["International_travel_controls"] = pd.DataFrame(df["C8_International travel controls"].apply(assign_label_international_travel_controls))


In [39]:
def assign_label_income_support(i): 
    if i == 2:
        return 'cover_more_than_half_income'
    if i == 1:
        return 'cover_less_than_half_income'
    if i == 0:
        return 'no_income_support'
new_df["Income_support"] = pd.DataFrame(df["E1_Income support"].apply(assign_label_income_support))


In [40]:
def assign_label_Debt_contract_relie(i): 
    if i == 2:
        return 'broad_relief'
    if i == 1:
        return 'narrow_relief'
    if i == 0:
        return 'no_relie'
new_df["Debt_contract_relie"] = pd.DataFrame(df["E2_Debt/contract relief"].apply(assign_label_Debt_contract_relie))


In [41]:
def assign_label_Public_information_campaigns(i): 
    if i == 2:
        return 'coordinated_public_information_campaign'
    if i == 1:
        return 'public_officials_urging_caution'
    if i == 0:
        return 'no_campaign'
new_df["Public_information_campaigns"] = pd.DataFrame(df["H1_Public information campaigns"].apply(assign_label_Public_information_campaigns))


In [42]:
def assign_label_Testing_policy(i): 
    if i == 3:
        return 'open_public_testing'
    if i == 2:
        return 'test_anyone_show_symptoms'
    if i == 1:
        return 'test_selected_ppl'
    if i == 0:
        return 'no_testing_policy'
new_df["Testing_policy"] = pd.DataFrame(df["H2_Testing policy"].apply(assign_label_Testing_policy))


In [43]:
def assign_label_Contact_tracing(i):     
    if i == 2:
        return 'comprehensive_tracing'
    if i == 1:
        return 'limited_tracing'
    if i == 0:
        return 'no_contact_tracing'
new_df["Contact_tracing"] = pd.DataFrame(df["H3_Contact tracing"].apply(assign_label_Contact_tracing))


In [44]:
def assign_label_StringencyIndex(i):
    if i >70:
        return 'gov_stringency_very_high'
    if i >60 and i<= 70:
        return 'gov_stringency_high'
    if i >40 and i<= 60:
        return 'gov_stringency_medium'
    if i >20 and i<= 40:
        return 'gov_stringency_low'
    if i >0 and i<= 20:
        return 'gov_stringency_very_low'
    if i == 0:
        return 'gov_stringency_none'
new_df["Gov_stringency_Index"] = pd.DataFrame(df["StringencyIndex"].apply(assign_label_StringencyIndex))


In [45]:
def assign_label_StringencyIndex(i):
    if i >70:
        return 'gov_stringency_very_high'
    if i >60 and i<= 70:
        return 'gov_stringency_high'
    if i >40 and i<= 60:
        return 'gov_stringency_medium'
    if i >20 and i<= 40:
        return 'gov_stringency_low'
    if i >0 and i<= 20:
        return 'gov_stringency_very_low'
    if i == 0:
        return 'gov_stringency_none'
new_df["Gov_stringency_Index"] = pd.DataFrame(df["StringencyIndex"].apply(assign_label_StringencyIndex))


In [46]:
def assign_label_GovernmentResponse(i):
    if i >70:
        return 'Government_Response_very_high'
    if i >60 and i<= 70:
        return 'Government_Response_high'
    if i >40 and i<= 60:
        return 'Government_Response_medium'
    if i >20 and i<= 40:
        return 'Government_Response_low'
    if i >0 and i<= 20:
        return 'Government_Response_very_low'
    if i == 0:
        return 'Government_Response_none'
new_df["Government_Response"] = pd.DataFrame(df["GovernmentResponseIndex"].apply(assign_label_GovernmentResponse))


In [47]:
def assign_label_ContainmentHealth(i):
    if i >70:
        return 'ContainmentHealth_very_high'
    if i >60 and i<= 70:
        return 'ContainmentHealth_high'
    if i >40 and i<= 60:
        return 'ContainmentHealth_medium'
    if i >20 and i<= 40:
        return 'ContainmentHealth_low'
    if i >0 and i<= 20:
        return 'ContainmentHealth_very_low'
    if i == 0:
        return 'ContainmentHealth_none'
new_df["Containment_Health"] = pd.DataFrame(df["ContainmentHealthIndex"].apply(assign_label_ContainmentHealth))


In [48]:
def assign_label_Employed_total(i):
    if i >= 13000:
        return 'total_employed_very_high'
    if i >= 12800 and i < 13000:
        return 'total_employed_high'
    if i >= 12600 and i < 12800:
        return 'total_employed_medium'
    if i >= 12400 and i < 12600:
        return 'total_employed_low'
    if i >= 12200 and i < 12400:
        return 'total_employed_very_low'
    if i < 12200:
        return 'total_employed_extreme_low'      

new_df["Employed_total"] = pd.DataFrame(df["Employed_total"].apply(assign_label_Employed_total))


In [49]:
def assign_label_Full_time(i):
    if i >= 8900:
        return 'Full_time_employed_very_high'
    if i >= 8800 and i < 8900:
        return 'Full_time_employed_high'
    if i >= 8700 and i < 8800:
        return 'Full_time_employed_medium'
    if i >= 8600 and i < 8700:
        return 'Full_time_employed_low'
    if i >= 8500 and i < 8600:
        return 'Full_time_employed_very_low'
    if i < 8500:
        return 'Full_time_employed_extreme_low'      

new_df["Full_time"] = pd.DataFrame(df["Full_time"].apply(assign_label_Full_time))


In [50]:
def assign_label_Part_time(i):
    if i >= 4100:
        return 'Part_time_employed_very_high'
    if i >= 4000 and i < 4100:
        return 'Part_time_employed_high'
    if i >= 3900 and i < 4000:
        return 'Part_time_employed_medium'
    if i >= 3800 and i < 3900:
        return 'Part_time_employed_low'
    if i >= 3700 and i < 3800:
        return 'Part_time_employed_very_low'
    if i < 3700:
        return 'Part_time_employed_extreme_low'      

new_df["Part_time"] = pd.DataFrame(df["Part_time"].apply(assign_label_Part_time))


In [51]:
def assign_label_Unemployment_rate(i):
    if i >= 7:
        return 'Unemployment_rate_very_high'
    if i >= 6 and i < 7:
        return 'Unemployment_rate_high'
    if i >= 5.5 and i < 6:
        return 'Unemployment_rate_medium'
    if i >= 5 and i < 5.5:
        return 'Unemployment_rate_low'
    if i < 5:
        return 'Unemployment_rate_very_low'         

new_df["Unemployment_rate"] = pd.DataFrame(df["Unemployment_rate"].apply(assign_label_Unemployment_rate))


In [52]:
print(new_df)

   NewCasesMonthlyChange NewDeathsMonthlyChange                School_closing  \
0               no_cases              no_deaths                 School_normal   
1               no_cases              no_deaths                 School_normal   
2               no_cases              no_deaths                 School_normal   
3               no_cases              no_deaths                 School_normal   
4               no_cases              no_deaths                 School_normal   
5               no_cases              no_deaths                 School_normal   
6               no_cases              no_deaths                 School_normal   
7               no_cases              no_deaths                 School_normal   
8               no_cases              no_deaths                 School_normal   
9               no_cases              no_deaths                 School_normal   
10              no_cases              no_deaths                 School_normal   
11              no_cases    

In [53]:
df.iloc[:, 16:]

,ConfirmedDeaths,StringencyIndex,StringencyLegacyIndex,GovernmentResponseIndex,ContainmentHealthIndex,EconomicSupportIndex,Employed_total,Full_time,Part_time,Unemployment_rate,NewCasesMonthlyChange,NewDeathsMonthlyChange
0,0.0,0.00,0.00,0.00,0.00,0.0,12603.100,8738.00,3865.100,5.50,0.0,0.0
1,0.0,0.00,0.00,0.00,0.00,0.0,12802.200,8828.30,3973.800,5.40,0.0,0.0
2,0.0,0.00,0.00,0.00,0.00,0.0,12785.200,8763.50,4021.800,5.40,0.0,0.0
3,0.0,0.00,0.00,0.00,0.00,0.0,12831.900,8746.40,4085.500,5.30,0.0,0.0
4,0.0,0.00,0.00,0.00,0.00,0.0,12920.900,8768.70,4152.100,5.10,0.0,0.0
5,0.0,0.00,0.00,0.00,0.00,0.0,12889.100,8783.00,4106.100,5.10,0.0,0.0
6,0.0,0.00,0.00,0.00,0.00,0.0,12884.600,8854.60,4030.000,5.20,0.0,0.0
7,0.0,0.00,0.00,0.00,0.00,0.0,12853.700,8745.30,4108.400,5.30,0.0,0.0
8,0.0,0.00,0.00,0.00,0.00,0.0,12919.800,8803.10,4116.800,5.10,0.0,0.0
9,0.0,0.00,0.00,0.00,0.00,0.0,12891.900,8795.10,4096.900,5.00,0.0,0.0


In [54]:
new_df.to_csv(r"D:\Download\COVID19\association_rule.csv", index = False, header=True)